# Prédiction de trajectoire de navires avec TensorFlow (multi-horizon)
Prédiction à 5, 10 et 15 minutes depuis le même point (15min avant la fin du trajet).

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go

In [25]:
# Fonctions utilitaires
def load_and_prepare_data(csv_path):
    df = pd.read_csv(csv_path, parse_dates=["BaseDateTime"])
    df.sort_values("BaseDateTime", inplace=True)
    df = df.dropna(subset=["LAT", "LON"])
    return df

def add_time_features(df):
    df = df.copy()
    df['hour'] = df['BaseDateTime'].dt.hour
    df['minute'] = df['BaseDateTime'].dt.minute
    return df

def add_previous_features(df):
    df = df.copy()
    df['SOG_prev'] = df.groupby('MMSI')['SOG'].shift(1)
    df['COG_prev'] = df.groupby('MMSI')['COG'].shift(1)
    df['LAT_prev'] = df.groupby('MMSI')['LAT'].shift(1)
    df['LON_prev'] = df.groupby('MMSI')['LON'].shift(1)
    return df

def prepare_features(df):
    df = add_time_features(df)
    df = add_previous_features(df)
    return df

def create_future_targets(df, horizons=[5,10,15]):
    df = df.sort_values(["MMSI", "BaseDateTime"]).copy()
    for h in horizons:
        df[f'LAT_t+{h}'] = df.groupby('MMSI')['LAT'].shift(-h)
        df[f'LON_t+{h}'] = df.groupby('MMSI')['LON'].shift(-h)
    return df

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    dlat = lat2 - lat1
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c
def train_keras_model(df_train, features, target_cols, epochs=50, verbose=0):
    scaler = StandardScaler()
    X = df_train[features].fillna(df_train[features].mean())
    X_scaled = scaler.fit_transform(X)
    y = df_train[target_cols].values
    model = keras.Sequential([
        keras.layers.Input(shape=(X_scaled.shape[1],)),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(50, activation='relu'),
        keras.layers.Dense(len(target_cols))
    ])
    model.compile(optimizer='adam', loss='mae')
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # Passer verbose en argument pour contrôler l'affichage des epochs
    model.fit(X_scaled, y, epochs=epochs, verbose=verbose, validation_split=0.2, callbacks=[callback])
    return model, scaler

def get_15min_before_end_points(df, horizon=15):
    points = []
    for mmsi, group in df.groupby("MMSI"):
        group_sorted = group.sort_values("BaseDateTime")
        last_time = group_sorted["BaseDateTime"].iloc[-1]
        target_time = last_time - pd.Timedelta(minutes=horizon)
        idx = group_sorted[group_sorted["BaseDateTime"] <= target_time].index
        if len(idx) == 0:
            idx = [group_sorted.index[-2]] if len(group_sorted) > 1 else [group_sorted.index[-1]]
        else:
            idx = [idx[-1]]
        start_row = group_sorted.loc[idx]
        end_row = group_sorted.iloc[[-1]]
        points.append({
            "MMSI": mmsi,
            "start_idx": start_row.index[0],
            "BaseDateTime_start": start_row["BaseDateTime"].values[0]
        })
    return pd.DataFrame(points)

In [26]:
# Chargement et préparation des données
csv_path = "Besoin3-Gauthier/After_Sort.csv"
horizons = [5, 10, 15]

df = load_and_prepare_data(csv_path)
df = prepare_features(df)
df = create_future_targets(df, horizons=horizons)

# Split MMSI (80% train, 20% test)
mmsi_uniques = df['MMSI'].dropna().unique()
np.random.seed(42)
np.random.shuffle(mmsi_uniques)
n_train = int(0.8 * len(mmsi_uniques))
mmsi_train = mmsi_uniques[:n_train]
mmsi_test = mmsi_uniques[n_train:]
df_train = df[df['MMSI'].isin(mmsi_train)]
df_test = df[df['MMSI'].isin(mmsi_test)]

In [ ]:
# Définition des features et entraînement des modèles pour chaque horizon
features = [
    "LAT", "LON", "SOG", "COG", "Heading", "VesselType", "Length", "Draft",
    "hour", "minute","SOG_prev", "COG_prev", "LAT_prev", "LON_prev"
    ]

models = {}
scalers = {}
for h in horizons:
    target_cols = [f'LAT_t+{h}', f'LON_t+{h}']
    df_train_h = df_train.dropna(subset=features + target_cols)
    # Afficher la progression de l'entraînement (epochs) avec verbose=1
    model, scaler = train_keras_model(df_train_h, features, target_cols, epochs=50, verbose=1)
    models[h] = model
    scalers[h] = scaler

Epoch 1/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 8.0559 - val_loss: 0.2202
Epoch 2/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 8.0559 - val_loss: 0.2202
Epoch 2/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.1233 - val_loss: 0.1677
Epoch 3/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.1233 - val_loss: 0.1677
Epoch 3/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.1091 - val_loss: 0.0907
Epoch 4/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.1091 - val_loss: 0.0907
Epoch 4/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0982 - val_loss: 0.1082
Epoch 5/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0982 - val_loss: 0.1082
Epoch 5/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0963 - val_loss: 0.1201
Epoch 6/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0963 - val_loss: 0.1201
Epoch 6/50
4639/4639 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0918 - val_loss: 0.0880
Epoch 7/50
4639/4639 ━━━━━

In [ ]:
# Sélection du point de départ (15min avant la fin) pour chaque MMSI
# On limite à 40 derniers éléments du trajet pour chaque MMSI
def get_15min_before_end_points_limited(df, horizon=15, max_points=40):
    points = []
    for mmsi, group in df.groupby("MMSI"):
        group_sorted = group.sort_values("BaseDateTime")
        # On ne garde que les 40 derniers points (ou moins)
        group_sorted = group_sorted.iloc[-max_points:]
        last_time = group_sorted["BaseDateTime"].iloc[-1]
        target_time = last_time - pd.Timedelta(minutes=horizon)
        idx = group_sorted[group_sorted["BaseDateTime"] <= target_time].index
        if len(idx) == 0:
            idx = [group_sorted.index[-2]] if len(group_sorted) > 1 else [group_sorted.index[-1]]
        else:
            idx = [idx[-1]]
        start_row = group_sorted.loc[idx]
        end_row = group_sorted.iloc[[-1]]
        points.append({
            "MMSI": mmsi,
            "start_idx": start_row.index[0],
            "BaseDateTime_start": start_row["BaseDateTime"].values[0]
        })
    return pd.DataFrame(points)

points = get_15min_before_end_points_limited(df_test, horizon=15, max_points=40)



Description des points sélectionnés pour chaque MMSI (limités aux 40 derniers points du trajet) :


,MMSI,start_idx,BaseDateTime_start
count,3.000000e+01,30.000000,30
mean,4.363527e+08,397253.233333,2023-05-31 21:41:07.733333248
min,2.095130e+08,339706.000000,2023-05-31 07:01:33
25%,3.672382e+08,401549.000000,2023-05-31 22:23:04.750000128
50%,3.682370e+08,401719.500000,2023-05-31 23:42:40.500000
75%,5.380083e+08,401816.250000,2023-05-31 23:44:18.249999872
max,6.718300e+08,411595.000000,2023-05-31 23:44:56
std,1.205030e+08,15550.758480,NaN


In [ ]:
# Prédiction et évaluation pour chaque horizon
preds = []
for _, row in points.iterrows():
    start_idx = row["start_idx"]
    X = df_test.loc[[start_idx], features].fillna(df_train[features].mean())
    for h in horizons:
        X_scaled = scalers[h].transform(X)
        y_pred = models[h].predict(X_scaled, verbose=0)[0]
        lat_pred, lon_pred = y_pred[0], y_pred[1]
        lat_true = df_test.loc[start_idx, f'LAT_t+{h}'] if f'LAT_t+{h}' in df_test.columns else np.nan
        lon_true = df_test.loc[start_idx, f'LON_t+{h}'] if f'LON_t+{h}' in df_test.columns else np.nan
        dist = np.nan
        if not pd.isna(lat_true) and not pd.isna(lon_true):
            dist = haversine_distance(lat_pred, lon_pred, lat_true, lon_true)
        preds.append({
            "MMSI": row["MMSI"],
            "BaseDateTime_start": row["BaseDateTime_start"],
            "horizon": h,
            "LAT_pred": lat_pred,
            "LON_pred": lon_pred,
            "LAT_true": lat_true,
            "LON_true": lon_true,
            "distance_error_m": dist
        })
df_preds = pd.DataFrame(preds)
display(df_preds[["MMSI", "BaseDateTime_start", "horizon", "LAT_pred", "LON_pred", "LAT_true", "LON_true", "distance_error_m"]])
print(len(df_preds), "prédictions effectuées.\n")
print("\nErreur moyenne (mètres) pour chaque horizon (sur les cas où la vérité est connue) :")
display(df_preds.groupby("horizon")["distance_error_m"].mean())

,MMSI,BaseDateTime_start,horizon,LAT_pred,LON_pred,LAT_true,LON_true,distance_error_m
0,209513000,2023-05-31 23:44:25,5,26.156561,-80.352455,26.08809,-80.11621,24784.420670
1,209513000,2023-05-31 23:44:25,10,26.116337,-80.381493,26.08809,-80.11620,26676.131894
2,209513000,2023-05-31 23:44:25,15,26.109911,-80.229424,NaN,NaN,NaN
3,255805583,2023-05-31 23:44:56,5,29.008987,-95.471481,NaN,NaN,NaN
4,255805583,2023-05-31 23:44:56,10,28.922832,-95.334702,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
85,636021151,2023-05-31 23:28:40,10,26.697760,-79.658524,NaN,NaN,NaN
86,636021151,2023-05-31 23:28:40,15,26.608625,-79.185646,NaN,NaN,NaN
87,671830000,2023-05-31 23:40:38,5,25.897768,-80.565887,25.79638,-80.24543,33992.442507
88,671830000,2023-05-31 23:40:38,10,25.769325,-80.328957,NaN,NaN,NaN


90 prédictions effectuées.


Erreur moyenne (mètres) pour chaque horizon (sur les cas où la vérité est connue) :


horizon
5     72771.970667
10    29282.252906
15             NaN
Name: distance_error_m, dtype: float64

In [ ]:
# Visualisation : trajets réels + prédictions depuis le même point pour chaque bateau
fig = go.Figure()

# 1. Tracer la trajectoire réelle de chaque MMSI du jeu de test
for mmsi, group in df_test.groupby("MMSI"):
    group_sorted = group.sort_values("BaseDateTime")
    fig.add_trace(go.Scattermap(
        lat=group_sorted["LAT"],
        lon=group_sorted["LON"],
        mode="lines+markers",
        marker=dict(size=6, color="#636efa"),
        line=dict(color="#636efa", width=2),
        name=f"MMSI {mmsi} - Trajet réel",
        legendgroup=f"{mmsi}_reel",
        showlegend=False
    ))

# 2. Afficher les points de départ, prédictions et relier en pointillé, avec info détaillée sur la carte
color_map = {5: "#EF553B", 10: "#00CC96", 15: "#AB63FA"}
for _, row in points.iterrows():
    mmsi = row["MMSI"]
    start_idx = row["start_idx"]
    start_lat = df_test.loc[start_idx, "LAT"]
    start_lon = df_test.loc[start_idx, "LON"]
    start_time = df_test.loc[start_idx, "BaseDateTime"]
    # Point de départ avec info détaillée
    start_text = (
        f"Départ<br>MMSI: {mmsi}<br>Heure: {start_time}<br>Lat: {start_lat:.5f}<br>Lon: {start_lon:.5f}"
    )
    fig.add_trace(go.Scattermap(
        lat=[start_lat],
        lon=[start_lon],
        mode="markers+text",
        marker=dict(size=12, color="black", symbol="circle"),
        name="Point de départ",
        legendgroup="start_point",
        showlegend=False,
        text=[start_text],
        textposition="top right",
        textfont=dict(color="black", size=11)
    ))
    # Prédictions pour chaque horizon
    for h in horizons:
        pred_row = df_preds[(df_preds["MMSI"] == mmsi) & (df_preds["BaseDateTime_start"] == row["BaseDateTime_start"]) & (df_preds["horizon"] == h)]
        if pred_row.empty:
            continue
        pred_row = pred_row.iloc[0]
        pred_time = pd.to_datetime(start_time) + pd.Timedelta(minutes=h)
        lat_pred = pred_row["LAT_pred"]
        lon_pred = pred_row["LON_pred"]
        dist = pred_row["distance_error_m"]
        # Info détaillée pour le point prédit
        pred_text = (
            f"Prédiction +{h}min<br>MMSI: {mmsi}<br>Heure: {pred_time}<br>Lat: {lat_pred:.5f}<br>Lon: {lon_pred:.5f}"
        )
        if not pd.isna(dist):
            pred_text += f"<br>Erreur: {dist:.1f} m"
        else:
            pred_text += "<br>Erreur: N/A"
        fig.add_trace(go.Scattermap(
            lat=[lat_pred],
            lon=[lon_pred],
            mode="markers+text",
            marker=dict(size=14, color=color_map[h], symbol="star"),
            name=f"Prédiction +{h}min",
            legendgroup=f"pred_{h}",
            showlegend=False,
            text=[pred_text],
            textposition="top left",
            textfont=dict(color=color_map[h], size=11)
        ))
        # Ligne pointillée entre point de départ et prédiction
        fig.add_trace(go.Scattermap(
            lat=[start_lat, lat_pred],
            lon=[start_lon, lon_pred],
            mode="lines",
            line=dict(color=color_map[h], width=2),
            name=f"Départ→Prédiction +{h}min",
            legendgroup=f"predline_{h}",
            showlegend=False
        ))

# 3. Ajouter des traces vides pour la légende
for h in horizons:
    fig.add_trace(go.Scattermap(
        lat=[None], lon=[None],
        mode="markers", marker=dict(size=14, color=color_map[h], symbol="star"),
        name=f"Prédiction +{h}min"
    ))
    fig.add_trace(go.Scattermap(
        lat=[None, None], lon=[None, None],
        mode="lines", line=dict(color=color_map[h], width=2),
        name=f"Départ→Prédiction +{h}min"
    ))
fig.add_trace(go.Scattermap(
    lat=[None], lon=[None],
    mode="markers", marker=dict(size=12, color="black", symbol="circle"),
    name="Point de départ"
))
fig.add_trace(go.Scattermap(
    lat=[None], lon=[None],
    mode="lines+markers",
    marker=dict(size=6, color="#636efa"),
    line=dict(color="#636efa", width=2),
    name="Trajectoire réelle"
))

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center={"lat": df_test["LAT"].mean(), "lon": df_test["LON"].mean()},
    title="Trajectoires réelles et prédictions (+5min, +10min, +15min) depuis le même point pour chaque bateau (infos détaillées)",
    legend=dict(itemsizing="constant")
)
fig.show()

## Conseils pour améliorer la qualité des prédictions

- **Nettoyer les données** : Vérifiez qu'il n'y a pas d'anomalies ou de valeurs aberrantes dans les positions, vitesses, etc.
- **Ajouter des features** : Ajoutez des variables pertinentes (ex : différence de temps entre points, accélération, ports proches, météo si disponible).
- **Augmenter la taille du jeu d'entraînement** : Plus de données réelles améliorent la généralisation.
- **Normalisation/Standardisation** : Vérifiez que toutes les features sont bien normalisées.
- **Architecture du modèle** : Essayez d'augmenter la complexité du modèle (plus de couches, plus de neurones) ou testez d'autres architectures (LSTM, GRU pour les séries temporelles).
- **Hyperparamètres** : Ajustez le nombre d'epochs, le batch_size, le learning rate, la patience de l'early stopping.
- **Sélection des points d'entraînement** : Utilisez plus de points par trajet, pas seulement un point par bateau.
- **Validation croisée** : Utilisez la cross-validation pour évaluer la robustesse.
- **Analyse des erreurs** : Visualisez les cas où l'erreur est très grande pour comprendre les situations problématiques.
- **Entraînement séparé par type de navire** : Si les comportements sont très différents selon le type, entraînez un modèle par type.

N'hésitez pas à tester plusieurs de ces pistes pour voir ce qui améliore le plus vos résultats.